In [27]:
# cnn model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical

In [3]:
# load a single file as a numpy array
def load_file(filepath):
 dataframe = read_csv(filepath, header=None, delim_whitespace=True)
 return dataframe.values

In [23]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
 loaded = list()
 for name in filenames:
  data = load_file(prefix + name)
  loaded.append(data)
 #print('------before_dstack------')   
 #print(loaded) 
 # stack group so that features are the 3rd dimension
 loaded = dstack(loaded)
 #print('------after_dstack------')
 #print(loaded)
 return loaded

In [5]:
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
 filepath = prefix + group + '/Inertial Signals/'
 # load all 9 files as a single array
 filenames = list()
 # total acceleration
 filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
 # body acceleration
 filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
 # body gyroscope
 filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
 # load input data
 X = load_group(filenames, filepath)
 # load class output
 y = load_file(prefix + group + '/y_'+group+'.txt')
 return X, y

In [6]:
#load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
 # load all train
 trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
 print(trainX.shape, trainy.shape)
 # load all test
 testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
 print(testX.shape, testy.shape)
 # zero-offset class values ------?
 trainy = trainy - 1
 testy = testy - 1
 # one hot encode y -------?
 trainy = to_categorical(trainy)
 testy = to_categorical(testy)
 print(trainX.shape, trainy.shape, testX.shape, testy.shape)
 return trainX, trainy, testX, testy

In [29]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
 verbose, epochs, batch_size = 0, 10, 32
 n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
 model = Sequential()
 model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
 model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
 model.add(Dropout(0.5))
 model.add(MaxPooling1D(pool_size=2))
 model.add(Flatten())
 model.add(Dense(100, activation='relu'))
 model.add(Dense(n_outputs, activation='softmax'))
 model.copile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 # fit network
 model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
 # evaluate model
 _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
 return accuracy

In [8]:
# summarize scores
def summarize_results(scores):
 print(scores)
 m, s = mean(scores), std(scores)
 print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [19]:
# run an experiment
def run_experiment(repeats=10):
 # load data
 trainX, trainy, testX, testy = load_dataset()
 # repeat experiment
 scores = list()
 for r in range(repeats):
  score = evaluate_model(trainX, trainy, testX, testy)
  score = score * 100.0
  print('>#%d: %.3f' % (r+1, score))
  scores.append(score)
 # summarize results
 summarize_results(scores)

In [16]:
# run an experiment
def run_experiment(params, repeats=10):
 # load data
 trainX, trainy, testX, testy = load_dataset()
 # test each parameter
 all_scores = list()
 for p in params:
 # repeat experiment
  scores = list()
  for r in range(repeats):
   score = evaluate_model(trainX, trainy, testX, testy, p)
   score = score * 100.0
   print('>p=%s #%d: %.3f' % (p, r+1, score))
   scores.append(score)
   all_scores.append(scores)
 # summarize results
 summarize_results(all_scores, params)

In [30]:
# run the experiment
run_experiment()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 91.653
>#2: 90.159
>#3: 90.906
>#4: 90.261
>#5: 91.720
>#6: 88.565
>#7: 92.060
>#8: 92.399
>#9: 89.990
>#10: 91.381
[91.65253043174744, 90.15948176383972, 90.90600609779358, 90.26128053665161, 91.7203962802887, 88.5646402835846, 92.05971956253052, 92.39904880523682, 89.98982310295105, 91.3810670375824]
Accuracy: 90.909% (+/-1.109)


In [12]:
trainX, trainy, testX, testy = load_dataset()

------before_dstack------
[array([[1.012817 , 1.022833 , 1.022028 , ..., 1.018445 , 1.019372 ,
        1.021171 ],
       [1.018851 , 1.02238  , 1.020781 , ..., 1.014788 , 1.016499 ,
        1.017849 ],
       [1.023127 , 1.021882 , 1.019178 , ..., 1.021041 , 1.022935 ,
        1.022019 ],
       ...,
       [0.7548917, 0.8043137, 0.831714 , ..., 0.6956257, 0.7479103,
        0.776768 ],
       [0.9279268, 0.9129872, 0.9246597, ..., 0.6753473, 0.6603377,
        0.719353 ],
       [0.7980909, 0.8192417, 0.8658821, ..., 0.8980947, 0.8283723,
        0.8002428]]), array([[-0.1232167, -0.1268756, -0.1240037, ..., -0.1240696, -0.1227451,
        -0.121326 ],
       [-0.123976 , -0.1268078, -0.1277862, ..., -0.1290268, -0.1264244,
        -0.1305193],
       [-0.1200157, -0.1214994, -0.1228407, ..., -0.1308757, -0.1312099,
        -0.1301826],
       ...,
       [-0.1711829, -0.1999375, -0.2319457, ..., -0.2217656, -0.2811496,
        -0.3275988],
       [-0.223801 , -0.2210407, -0.1839923,

------before_dstack------
[array([[1.041216 , 1.041803 , 1.039086 , ..., 0.9930164, 0.9932414,
        0.9943906],
       [0.9991921, 0.9946787, 0.9935518, ..., 1.001861 , 0.9975208,
        0.9928615],
       [0.9975931, 0.9989703, 0.9970574, ..., 0.9918802, 0.9906626,
        0.9882446],
       ...,
       [0.8213505, 0.7991996, 0.8004623, ..., 1.46317  , 1.179223 ,
        0.8504963],
       [1.037668 , 0.8780725, 0.8963897, ..., 1.156389 , 1.243857 ,
        1.323546 ],
       [0.7713622, 0.9000949, 0.8681034, ..., 0.9188616, 0.9494752,
        0.9578348]]), array([[-0.2697959, -0.280025 , -0.2926631, ..., -0.2599865, -0.2620643,
        -0.2641348],
       [-0.2649349, -0.2532142, -0.2565887, ..., -0.2619359, -0.2713225,
        -0.2799715],
       [-0.2639912, -0.2638194, -0.2638495, ..., -0.2836712, -0.280597 ,
        -0.2822329],
       ...,
       [-0.2484623, -0.2232599, -0.179017 , ..., -0.5515283, -0.5472997,
        -0.4900368],
       [-0.3971532, -0.2848634, -0.2635297,